### Welcome to Week 5 Day 1

AutoGen AgentChat!

This should look simple and familiar, because it has a lot in common with Crew and OpenAI Agents SDK

In [7]:
from dotenv import load_dotenv
load_dotenv(override=True)

True

### First concept: the Model


The first concept then is the model. The model which is similar to concepts like LLM that we've had before. It's like a wrapper around calling a large language model and here we import something called `OpenAIChatCompletionClient`, which is the wrapper for the LLM we'll be using, which is `gpt-40mini`. This is how you create your model client as it's called, and it's very simple indeed. You just pass in the name of your model and run that.

Also, you can do the same thing with OLAMA to run a local model like `llama3.2`. It's exactly the same idea. You could run that and you could continue all of this in exactly the same way running locally instead of using `gpt-40mini`.



In [8]:
from autogen_ext.models.openai import OpenAIChatCompletionClient
model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")

In [9]:
from autogen_ext.models.ollama import OllamaChatCompletionClient
ollamamodel_client = OllamaChatCompletionClient(model="llama3.2")



### Second concept: The Message

The second concept is the message. This is something which is a different concept for AutoGen AgentChat. It's this idea that you create an object called text message, in this case, that has the content:
*"I'd like to go to London."*
This is my message right now. And the source is the user—me.

If we run that and print it, we see it's a text message, the source is the user, and there's the content. And now that is all there is to it. That is the message.


In [10]:
from autogen_agentchat.messages import TextMessage
message = TextMessage(content="I'd like to go to London", source="user")
message

TextMessage(source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 28, 18, 27, 15, 452651, tzinfo=datetime.timezone.utc), content="I'd like to go to London", type='TextMessage')

### Third concept: The Agent
The third concept is the agent, and it's very similar to things we've seen in the past. The thing that we import is called `AssistantAgent`. You'll see this many times. This is the most fundamental class that we'll work with in AutoGen AgentChat.

We create a new instance of `AssistantAgent`. We give it a name, `AirlineAgent`. We give it a model client, the underlying LLM. We give it a system message, rather like instructions in OpenAI:
*"You are a helpful assistant for an airline. You give a short, humorous answer."*

So let's give it that to see what that does to it. And `model_client_stream` is how we say that we want to stream back the results. This is something that we've already done from time to time. But that is an agent that has been created.

In [11]:
from autogen_agentchat.agents import AssistantAgent

agent = AssistantAgent(
    name="airline_agent",
    model_client=model_client,
    system_message="You are a helpful assistant for an airline. You give short, humorous answers.",
    model_client_stream=True
)

### Put it all together with on_messages

The thing that brings it all together is something called `onMessages`. That is what we call on an agent passing a bunch of messages, which we do right here. We just put our single message in a list. We pass that into `onMessages`. You also have to pass in a single cancellation token, which is how it knows when the messages are finished. That's the sort of fiddly thing about AgentChat, but I wouldn't worry about it. You just call. And, of course, it's async, so we have to await it:
`await agent.onMessages([message], cancellation_token)`

Then, print the `chatMessage.content`. So we're passing in this message: "I'd like to go to London." And the assistant is: "You're a helpful assistant for an airline. You give short, humorous answers."

Let's see what happens if we do this.
*Great choice. Just remember, if it starts raining, it's not a sign to panic. It's just London welcoming you.*

A nice, humorous answer from our agent.

It's worth pointing out that we're having the same kind of moment as with OpenAI Agents SDK. It's so easy to do this, to package it up and to make this call. It's really a nice, lightweight abstraction. There's not a lot of heaviness to this—just a lightweight abstraction around calling LLMs.

In [12]:
from autogen_core import CancellationToken

response = await agent.on_messages([message], cancellation_token=CancellationToken())
response.chat_message.content

'Great choice! London has everything: history, tea, and the occasional rain cloud. When do you want to jet off?'

### Let's make a local database of ticket prices

Well, let's take it a little bit further. Of course, we have to bring in tools. It's always about tools! Bring in a tool. Do something more interesting right now.

Now we're going to make a tool that's going to get ticket prices. We're going to arm our agent with the ability to look up ticket prices. And we might as well use a SQL-like database, because people often like to say, "OK, what would it be like if we had our agents being able to query the database?"

There are sophisticated ways of doing it, to actually write a SQL tool that gives agents the ability to write SQL. But in this case, it's perfectly simple. We can just write a tool that can just look up in the database.

So let's do that right now:

* Import `sqlite3`.
* Delete `tickets.db` if it already exists (because we ran this before).
* Once it's been deleted, connect to a new database and create a table called `cities`, which has a city name and a round-trip price.


In [13]:
import os
import sqlite3

# Delete existing database file if it exists
if os.path.exists("tickets.db"):
    os.remove("tickets.db")

# Create the database and the table
conn = sqlite3.connect("tickets.db")
c = conn.cursor()
c.execute("CREATE TABLE cities (city_name TEXT PRIMARY KEY, round_trip_price REAL)")
conn.commit()
conn.close()

People that are familiar with this will know perfectly well that it's created a `tickets.db` database that will now be empty. We are going to populate our database with a bunch of tickets to London, Paris, Rome, Madrid, Barcelona, and Berlin. And that's been done.

In [14]:
# Populate our database
def save_city_price(city_name, round_trip_price):
    conn = sqlite3.connect("tickets.db")
    c = conn.cursor()
    c.execute("REPLACE INTO cities (city_name, round_trip_price) VALUES (?, ?)", (city_name.lower(), round_trip_price))
    conn.commit()
    conn.close()

# Some cities!
save_city_price("London", 299)
save_city_price("Paris", 399)
save_city_price("Rome", 499)
save_city_price("Madrid", 550)
save_city_price("Barcelona", 580)
save_city_price("Berlin", 525)

We're going to write a simple query function, `getCityPrice`, that takes a name. It will get the round-trip price to travel to the city. It will simply connect to the database and run a little SELECT statement, passing in `cityName.lower`, and return the result.

Yes, the security cautious people, there's perhaps more things that I should do to make sure to validate this and make sure the city name is a city name and all the rest of it. But this is just a toy example for now.

In [15]:
# Method to get price for a city
def get_city_price(city_name: str) -> float | None:
    """ Get the roundtrip ticket price to travel to the city """
    conn = sqlite3.connect("tickets.db")
    c = conn.cursor()
    c.execute("SELECT round_trip_price FROM cities WHERE city_name = ?", (city_name.lower(),))
    result = c.fetchone()
    conn.close()
    return result[0] if result else None

So we run this. We've got `getCityPrice`. Let's just check it out:

* Try `getCityPrice('London')`. It's populated the database. We get back 299.
* Try `getCityPrice('Rome')`. We get back 499.

That does appear to be working.

In [16]:
get_city_price("Rome")

499.0

In [17]:
from autogen_agentchat.agents import AssistantAgent

smart_agent = AssistantAgent(
    name="smart_airline_agent",
    model_client=model_client,
    system_message="You are a helpful assistant for an airline. You give short, humorous answers, including the price of a roundtrip ticket.",
    model_client_stream=True,
    tools=[get_city_price],
    reflect_on_tool_use=True
)

In [18]:
response = await smart_agent.on_messages([message], cancellation_token=CancellationToken())
for inner_message in response.inner_messages:
    print(inner_message.content)
response.chat_message.content

[FunctionCall(id='call_w2owi8PMGmXCIY01aBNa5NzZ', arguments='{"city_name":"London"}', name='get_city_price')]
[FunctionExecutionResult(content='299.0', name='get_city_price', call_id='call_w2owi8PMGmXCIY01aBNa5NzZ', is_error=False)]


'A roundtrip ticket to London will set you back $299! Just enough to buy a cup of tea and some biscuits... or maybe just a single biscuit. Cheers!'

Very droll. There we go. So, a great, great answer, but of course more important than the great answer, is the fact that it was so simple to write that tool, to have it make a SQL call to a database that's sitting right there, and to have that run, use the tool, and it just shows, first of all, how quick and simple agent chat is, but secondly, how good you're getting at understanding this stuff, because honestly, this is so familiar to you now, that this is just in less than 10 minutes, you're already an expert at agent chat.